In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

only = gp.Model("only")
only.setParam("MIPGap", 1e-7)

x_ind = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x_individual")
yp_ind = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus_individual")
ym_ind = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus_individual")

M = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")

only.update()

obj = gp.quicksum(P_DA[t] * x_ind[i, t] for i in range(I) for t in range(T)) + gp.quicksum(1 / S * (P_RT[t, s] * yp_ind[i, t, s] - P_PN[t] * ym_ind[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    only.addConstr(R[i, t, s] - x_ind[i, t] == yp_ind[i, t, s] - ym_ind[i, t, s])
    only.addConstr(yp_ind[i, t, s] <= R[i, t, s])
    only.addConstr(yp_ind[i, t, s] <= M * z[i, t, s])
    only.addConstr(ym_ind[i, t, s] <= M * (1 - z[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {only.objVal}")
else:
    print("No optimal solution found.")
    
x_ind_vals = np.array([[x_ind[i, t].x for t in range(T)] for i in range(I)])
y_plus_ind_vals = np.array([[[yp_ind[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
y_minus_ind_vals = np.array([[[ym_ind[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=722.00, M2=1957.00
Set parameter MIPGap to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  1e-07

Optimize a model with 14400 rows, 10920 columns and 28800 nonzeros
Model fingerprint: 0x217419f9
Variable types: 7320 continuous, 3600 integer (3600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective 1183973.1877
Presolve removed 14310 rows and 10829 columns
Presolve time: 0.06s
Presolved: 90 rows, 91 columns, 210 nonzeros
Found heuristic solution: objective 1543704.3548
Variable types: 61 continuous, 30 integer (30 binary)

Root r

In [66]:
set = gp.Model("Settlement") 
set.setParam("PoolSolutions", 100)
set.setParam("PoolSearchMode", 2)
set.setParam("PoolGap", 0.05)

a = set.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
# x = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="x")
x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M_y = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
z_y = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")
d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
M_e = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z_e = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(1/S * (P_RT[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))
set.setObjective(obj, GRB.MAXIMIZE)

for i, t, s in product(range(I), range(T), range(S)):
    # set.addConstr(R[i, t, s] - x[i, t, s] == yp[i, t, s] - ym[i, t, s])
    set.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
    set.addConstr(yp[i, t, s] <= R[i, t, s])
    set.addConstr(yp[i, t, s] <= M_y * z_y[i, t, s])
    set.addConstr(ym[i, t, s] <= M_y * (1 - z_y[i, t, s]))
    set.addConstr(ep[i, t, s] == yp[i, t, s] - gp.quicksum(d[i, j, t, s] for j in range(I) if j != i))
    set.addConstr(em[i, t, s] == ym[i, t, s] - gp.quicksum(d[j, i, t, s] for j in range(I) if j != i))
    set.addConstr(gp.quicksum(d[i, j, t, s] for j in range(I) if j != i) <= yp[i, t, s])
    set.addConstr(gp.quicksum(d[j, i, t, s] for j in range(I) if j != i) <= ym[i, t, s])
    set.addConstr(d[i, i, t, s] == 0)
    # set.addConstr(x[i, t, s] >= x_ind_vals[i, t])
    set.addConstr(x[i, t] >= x_ind_vals[i, t])

for t, s in product(range(T), range(S)):
    # set.addConstr(a[t] == gp.quicksum(x[i, t, s] for i in range(I)))
    set.addConstr(a[t] == gp.quicksum(x[i, t] for i in range(I)))
    set.addConstr(bp[t, s] == gp.quicksum(ep[i, t, s] for i in range(I)))
    set.addConstr(bm[t, s] == gp.quicksum(em[i, t, s] for i in range(I)))
    # set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t, s] for i in range(I))
    #     == gp.quicksum(ep[i, t, s] for i in range(I)) - gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t] for i in range(I))
        == gp.quicksum(ep[i, t, s] for i in range(I)) - gp.quicksum(em[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))
    set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= M_e * z_e[t, s])
    set.addConstr(gp.quicksum(em[i, t, s] for i in range(I)) <= M_e * (1 - z_e[t, s]))

set.optimize()

if set.status == GRB.OPTIMAL:
    alpha_vals = np.array([a[t].x for t in range(T)])
    beta_plus_vals = np.array([[bp[t, s].x for s in range(S)] for t in range(T)])
    beta_minus_vals = np.array([[bm[t, s].x for s in range(S)] for t in range(T)])
    # x_vals = np.array([[[x[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array([[[yp[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    y_minus_vals = np.array([[[ym[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    d_vals = np.array([[[[d[i, j, t, s].x for s in range(S)] for t in range(T)] for j in range(I)] for i in range(I)])
    e_plus_vals = np.array([[[ep[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    e_minus_vals = np.array([[[em[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    given_vals = np.array([[[sum(d[i, j, t, s].x for j in range(I) if j != i) for s in range(S)] for t in range(T)] for i in range(I)])
    received_vals = np.array([[[sum(d[j, i, t, s].x for j in range(I) if j != i) for s in range(S)] for t in range(T)] for i in range(I)])
    print("\n- - - - - - - - - - - - - - - - - - - - - - - -")
    print("Optimal solution found!")
    print(set.objVal)
else:
    print("\nNo optimal solution found.")

Set parameter PoolSolutions to value 100
Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0.05
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  100
PoolSearchMode  2
PoolGap  0.05

Optimize a model with 41040 rows, 38304 columns and 151200 nonzeros
Model fingerprint: 0x6db136c0
Variable types: 33984 continuous, 4320 integer (4320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 30235 rows and 21360 columns
Presolve time: 0.13s
Presolved: 10805 rows, 16944 columns, 48245 nonzeros
Variable types: 12912 continuous, 4032 integer (4032 binary)
Found heuristic solution: objective 807802.83545

Root relaxation: objective 1.567562e+06, 6381 iterations, 0.04 seconds (0.0

In [67]:
solution_count = set.SolCount
print(f"총 {solution_count}개의 해가 저장되었습니다.")

# 해별 x[i, t]와 objective 값 저장
x_all_solutions = np.zeros((solution_count, I, T))
obj_values = np.zeros(solution_count)

# 각 해에서 값 저장
for k in range(solution_count):
    set.setParam(GRB.Param.SolutionNumber, k)
    obj_values[k] = set.PoolObjVal
    for i in range(I):
        for t in range(T):
            x_all_solutions[k, i, t] = x[i, t].Xn

# 값이 다른 x[i,t] 출력 + 해당 목적함수값도 함께 출력
diff_found = False
for i in range(I):
    for t in range(T):
        values = x_all_solutions[:, i, t]
        if not np.allclose(values, values[0], atol=1e-6):
            print(f"⚠️ x[{i},{t}] 값이 해마다 다릅니다: {values}")
            print(f"  ↳ 해당 해들의 objective 값: {obj_values}")
            diff_found = True

if not diff_found:
    print("✅ 모든 해에서 x[i,t] 값이 동일합니다.")
    print(f"  ↳ objective 값: {obj_values[0]}")

총 100개의 해가 저장되었습니다.
✅ 모든 해에서 x[i,t] 값이 동일합니다.
  ↳ objective 값: 1567561.8840047552


x_ind vs x 비교

In [45]:
rows = []
for t, i in product(range(9,11), range(I)):
        val_ind = x_ind_vals[i, t]
        val_set = x_vals[i, t]
        diff = val_set - val_ind
        rows.append([t, i, val_ind, val_set, diff])

df_x_compare = pd.DataFrame(rows, columns=["시간", "참가자", "x_ind", f"x", "diff"])
df_x_compare

,시간,참가자,x_ind,x,diff
0,9,0,3.0,6.0,3.0
1,9,1,39.0,41.0,2.0
2,9,2,51.0,58.0,7.0
3,9,3,11.0,18.0,7.0
4,9,4,10.0,20.0,10.0
5,10,0,19.0,35.0,16.0
6,10,1,35.0,35.0,0.0
7,10,2,56.0,80.0,24.0
8,10,3,31.0,36.0,5.0
9,10,4,20.0,113.0,93.0


ind vs ind(agg 참여 버젼) 수익 비교

In [65]:
profit_only = np.zeros(I)
profit_set = np.zeros(I)
internal_profit = np.zeros(I)
external_profit = np.zeros(I)

for i in range(I):
    # only 모델 수익
    da_only = sum(P_DA[t] * x_ind_vals[i, t] for t in range(T))
    rt_only = sum(P_RT[t, s] * y_plus_ind_vals[i, t, s] - P_PN[t] * y_minus_ind_vals[i, t, s] for t in range(T) for s in range(S)) / S
    profit_only[i] = da_only + rt_only
    
    # set 모델 수익
    da_set = sum(P_DA[t] * x[i, t].x for t in range(T))
    rt_set = sum(P_RT[t, s] * e_plus_vals[i, t, s] - P_PN[t] * e_minus_vals[i, t, s]
                 for t in range(T) for s in range(S)) / S
    profit_set[i] = da_set + rt_set
    
for i in range(I):
    # internal 정산 수익
    internal_out = sum(P_RT[t, s] * d_vals[i, j, t, s]
                       for j in range(I) if j != i
                       for t in range(T) for s in range(S)) / S
    internal_in  = sum(P_PN[t] * d_vals[j, i, t, s]
                       for j in range(I) if j != i
                       for t in range(T) for s in range(S)) / S
    internal_profit[i] = -internal_out + internal_in

    # external 시장 수익
    da = sum(P_DA[t] * x[i, t].x for t in range(T))
    rt = sum(P_RT[t, s] * yp[i, t, s].x - P_PN[t] * ym[i, t, s].x
             for t in range(T) for s in range(S)) / S
    external_profit[i] = da + rt

# 합계 계산
internal_sum = internal_profit.sum()
external_sum = external_profit.sum()
total_sum = internal_sum + external_sum

df_profit_full = pd.DataFrame({
    "참가자": range(I),
    "Profit_only": profit_only.round(2),
    "Profit_set": profit_set.round(2),
    "Internal": internal_profit.round(2),
    "External": external_profit.round(2),
    "Difference": (profit_set - profit_only).round(2)
})

# 마지막 줄에 총합 추가
df_profit_full.loc["합계"] = [
    "총합",
    round(profit_only.sum(), 2),
    round(profit_set.sum(), 2),
    round(internal_profit.sum(), 2),
    round(external_profit.sum(), 2),
    round(profit_set.sum() - profit_only.sum(), 2)
]

df_profit_full


,참가자,Profit_only,Profit_set,Internal,External,Difference
0,0,201827.57,200352.30,-168.79,200521.09,-1475.27
1,1,331310.50,332541.89,1686.36,330855.53,1231.39
2,2,406246.12,406388.03,539.12,405848.91,141.91
3,3,442248.26,447229.34,5248.73,441980.62,4981.08
4,4,162093.93,181050.32,27345.09,153705.22,18956.39
합계,총합,1543726.38,1567561.88,34650.51,1532911.37,23835.50


예시

In [71]:
rows = []

for s in range(0,1):
    for t in range(9,11):
        for i in range(I):
            R_val = R[i, t, s]
            x_val = x[i, t].x
            yp_val = yp[i, t, s].x
            ym_val = ym[i, t, s].x

            given = sum(d_vals[i, j, t, s] for j in range(I) if j != i)
            received = sum(d_vals[j, i, t, s] for j in range(I) if j != i)

            e_plus = yp_val - given
            e_minus = ym_val - received

            rows.append({
                "t": t,
                "s": s,
                "참가자": i,
                "R": round(R_val, 2),
                "x": round(x_val, 2),
                "y⁺": round(yp_val, 2),
                "y⁻": round(ym_val, 2),
                "given_d": round(given, 2),
                "received_d": round(received, 2),
                "e⁺": round(e_plus, 2),
                "e⁻": round(e_minus, 2),
            })

df_transaction_full = pd.DataFrame(rows)
df_transaction_full

,t,s,참가자,R,x,y⁺,y⁻,given_d,received_d,e⁺,e⁻
0,9,0,0,3,6.0,0.0,3.0,0.0,3.0,0.0,0.0
1,9,0,1,83,41.0,42.0,0.0,18.0,0.0,24.0,0.0
2,9,0,2,45,58.0,0.0,13.0,0.0,13.0,0.0,0.0
3,9,0,3,29,18.0,11.0,0.0,11.0,0.0,0.0,0.0
4,9,0,4,7,20.0,0.0,13.0,0.0,13.0,0.0,0.0
5,10,0,0,55,35.0,20.0,0.0,20.0,0.0,0.0,0.0
6,10,0,1,108,35.0,73.0,0.0,0.0,0.0,73.0,0.0
7,10,0,2,322,80.0,242.0,0.0,51.0,0.0,191.0,0.0
8,10,0,3,59,36.0,23.0,0.0,0.0,0.0,23.0,0.0
9,10,0,4,42,113.0,0.0,71.0,0.0,71.0,0.0,0.0
